In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
import xarray
import tsdm
import numpy as np
from zipfile import ZipFile

from tqdm import tqdm, trange
from pathlib import Path
import pandas
from pandas import DataFrame

In [ ]:
file_path = Path.cwd().parent.joinpath("data/some_data_from_db.pk.zip")
zipfile = ZipFile(file_path, "r")
content = zipfile.namelist()

In [ ]:
data: dict[int, list[dict[["measurements", "meta"], DataFrame]]] = pickle.load(
    zipfile.open(content[0])
)

In [ ]:
{i: len(x) for i, x in data.items()}

In [ ]:
metadata = pandas.concat(x["meta"] for x in data[8]).set_index("experiment_id")

In [ ]:
pandas.isna(
    data[1][0]["measurements"].set_index(["measurement_time", "measurement_id"])
)

In [ ]:
dsets = []

for x in tqdm(data[1]):
    measurements, metadata = x["measurements"], x["meta"]
    #     measurements = measurements.set_index(["measurement_time"])
    measurements["unit"] = measurements["unit"].astype(pandas.StringDtype())
    measurements = measurements.to_xarray()
    measurements = measurements.assign_coords(
        experiment_id=metadata["experiment_id"][0]
    )
    measurements = measurements.to_dataframe()
    #     metadata = metadata.set_index("experiment_id")
    dsets.append(measurements)
#     xarray.concat(dsets, dim="aga")

In [ ]:
measurements

In [ ]:
df = pandas.concat(dsets).set_index(
    [
        "experiment_id",
        "measurement_time",
    ]
)
df["unit"] = df["unit"].astype("category")
df.to_hdf(
    Path.cwd().parent.joinpath("data/some_data_from_db.h5"),
    key="data",
    format="table",
    mode="w",
)

In [ ]:
pandas.read_hdf(Path.cwd().parent.joinpath("data/some_data_from_db.h5"))